## Predicting Red Hat Business Value

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from datetime import datetime
%matplotlib inline

train = pd.read_csv("../../../Kaggle Data/Red Hat/Data/act_train.csv")
test = pd.read_csv("../../../Kaggle Data/Red Hat/Data/act_test.csv")
people = pd.read_csv("../../../Kaggle Data/Red Hat/Data/people.csv")

In [2]:
train.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


In [3]:
test.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10
0,ppl_100004,act1_249281,2022-07-20,type 1,type 5,type 10,type 5,type 1,type 6,type 1,type 1,type 7,type 4,NaN
1,ppl_100004,act2_230855,2022-07-20,type 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 682
2,ppl_10001,act1_240724,2022-10-14,type 1,type 12,type 1,type 5,type 4,type 6,type 1,type 1,type 13,type 10,NaN
3,ppl_10001,act1_83552,2022-11-27,type 1,type 20,type 10,type 5,type 4,type 6,type 1,type 1,type 5,type 5,NaN
4,ppl_10001,act2_1043301,2022-10-15,type 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 3015


In [4]:
people.head()

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,...,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,...,False,False,True,True,True,True,False,True,True,99
3,ppl_100004,type 2,group 22593,type 3,2022-07-20,type 40,type 25,type 9,type 4,type 16,...,True,True,True,True,True,True,True,True,True,76
4,ppl_100006,type 2,group 6534,type 3,2022-07-27,type 40,type 25,type 9,type 3,type 8,...,False,False,True,False,False,False,True,True,False,84


In [5]:
print(train.dtypes, "\n")
print(test.dtypes, "\n")
print(people.dtypes, "\n")

people_id            object
activity_id          object
date                 object
activity_category    object
char_1               object
char_2               object
char_3               object
char_4               object
char_5               object
char_6               object
char_7               object
char_8               object
char_9               object
char_10              object
outcome               int64
dtype: object 

people_id            object
activity_id          object
date                 object
activity_category    object
char_1               object
char_2               object
char_3               object
char_4               object
char_5               object
char_6               object
char_7               object
char_8               object
char_9               object
char_10              object
dtype: object 

people_id    object
char_1       object
group_1      object
char_2       object
date         object
char_3       object
char_4       object
char_5       obj

In [6]:
test_ids = test['activity_id']

def preprocess_acts(data, train_set = True):
    
    if train_set:
        data = data.drop(['outcome'], axis = 1)
    data['people_id'] = data['people_id'].apply(lambda x: x.split('_')[1])
    data['people_id'] = pd.to_numeric(data['people_id']).astype(int)
    data = add_date_fields(data)
    data = data.drop(['date','activity_id'], axis = 1)
    columns = list(data.columns)
    
    for col in columns[1:12]:
        data[col] = data[col].fillna('type -999')
        data[col] = data[col].apply(lambda x: x.split(' ')[1])
        data[col] = pd.to_numeric(data[col]).astype(int)
    for col in columns[12:15]:
        data[col] = pd.to_numeric(data[col]).astype(int)
    return data

def preprocess_people(data):
    data = data.drop(['date'], axis=1)
    data['people_id'] = data['people_id'].apply(lambda x: x.split('_')[1])
    data['people_id'] = pd.to_numeric(data['people_id']).astype(int)
    columns = list(data.columns)
    bools = columns[11:]
    strings = columns[1:11]

    for col in bools:
        data[col] = pd.to_numeric(data[col]).astype(int)
    for col in strings:
        data[col] = data[col].fillna('type -999')
        data[col] = data[col].apply(lambda x: x.split(' ')[1])
        data[col] = pd.to_numeric(data[col]).astype(int)
    return data

def add_date_fields(data): 
    data['date_year'] = data['date'].apply(lambda x: str(x.year))
    data['date_month'] = data['date'].apply(lambda x: str(x.month))
    data['date_day'] = data['date'].apply(lambda x: str(x.day))
    #data['date_weekday'] = data['date'].apply(lambda x: str(x.weekday))  
    return data

In [7]:
# convert field 'date' into data type
train['date'] = train['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d') )

test['date'] = test['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d') )

In [8]:
preprocess_people = preprocess_people(people)
actions_train = preprocess_acts(train)
actions_test = preprocess_acts(test, train_set = False)

In [9]:
# Training data
features = actions_train.merge(preprocess_people, how = 'inner', on = 'people_id')
labels = train['outcome']
#Testing data
test = actions_test.merge(preprocess_people, how = 'inner', on='people_id')

features.sample(10)

,people_id,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
764773,229547,5,-999,-999,-999,-999,-999,-999,-999,-999,...,0,0,0,0,0,0,0,0,0,71
1804665,392827,2,-999,-999,-999,-999,-999,-999,-999,-999,...,1,1,1,0,1,1,1,1,1,56
206023,129030,2,-999,-999,-999,-999,-999,-999,-999,-999,...,0,0,0,0,0,0,0,0,0,0
1320403,317042,2,-999,-999,-999,-999,-999,-999,-999,-999,...,0,0,0,0,0,0,0,0,1,95
450750,173509,2,-999,-999,-999,-999,-999,-999,-999,-999,...,1,1,1,0,1,1,1,1,1,56
1362385,324676,2,-999,-999,-999,-999,-999,-999,-999,-999,...,0,0,0,0,0,0,0,0,0,84
435815,170675,2,-999,-999,-999,-999,-999,-999,-999,-999,...,0,0,0,1,0,1,0,1,1,90
1320087,317022,2,-999,-999,-999,-999,-999,-999,-999,-999,...,0,0,0,0,0,0,0,0,0,61
1319154,316934,2,-999,-999,-999,-999,-999,-999,-999,-999,...,1,1,0,1,1,1,1,1,1,93
1805808,393042,2,-999,-999,-999,-999,-999,-999,-999,-999,...,0,0,0,0,0,0,0,0,0,89


In [10]:
#Split training set
from sklearn.cross_validation import train_test_split

num_test = 0.20
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = num_test, random_state = 30)
X_train.dtypes

people_id            int64
activity_category    int64
char_1_x             int64
char_2_x             int64
char_3_x             int64
char_4_x             int64
char_5_x             int64
char_6_x             int64
char_7_x             int64
char_8_x             int64
char_9_x             int64
char_10_x            int64
date_year            int64
date_month           int64
date_day             int64
char_1_y             int64
group_1              int64
char_2_y             int64
char_3_y             int64
char_4_y             int64
char_5_y             int64
char_6_y             int64
char_7_y             int64
char_8_y             int64
char_9_y             int64
char_10_y            int64
char_11              int64
char_12              int64
char_13              int64
char_14              int64
char_15              int64
char_16              int64
char_17              int64
char_18              int64
char_19              int64
char_20              int64
char_21              int64
c

### Random Forest

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

clf = RandomForestClassifier(n_estimators = 12, max_depth = 35, n_jobs = -1, warm_start = True )
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=35, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=12, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0, warm_start=True)

In [12]:
feature_importances = zip(X_train.columns, (clf.feature_importances_ * 100).astype(int))
sorted(feature_importances, key = lambda x: -x[1])[0:10]

[('char_38', 29),
 ('group_1', 16),
 ('char_2_y', 5),
 ('people_id', 4),
 ('char_7_y', 4),
 ('char_6_y', 3),
 ('date_month', 2),
 ('char_4_y', 2),
 ('char_8_y', 2),
 ('char_9_y', 2)]

Seems like 'char_38' is the most important feature of the whole dataset.

In [13]:
copy = X_train

In [21]:
features_columns = ["char_38","group_1","char_2_y","char_7_y","people_id","last_pred"]

In [14]:
X_train = copy[features_columns]

### Decision Tree Classifier

In [94]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [95]:
clf = DecisionTreeClassifier(criterion = 'entropy', max_depth = None, random_state = 1)

In [96]:
clf.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=1, splitter='best')

In [97]:
probability = clf.predict_proba(X_train)
predictions = probability[:,1]
score = roc_auc_score(y_train, predictions)
print("Area under ROC {0}".format(score))

Area under ROC 0.9975640246512064


In [98]:
probability = clf.predict_proba(X_test[features_columns])
predictions = probability[:,1]
score = roc_auc_score(y_test, predictions)
print("Area under ROC {0}".format(score))

Area under ROC 0.9954156627636637


In [99]:
test_probability = clf.predict_proba(test[features_columns])
test_predictions = test_probability[:,1]
output = pd.DataFrame({'activity_id': test_ids, 'outcome':test_predictions})
output.head()

,activity_id,outcome
0,act1_249281,0.0
1,act2_230855,0.0
2,act1_240724,1.0
3,act1_83552,1.0
4,act2_1043301,1.0


### ADA Boost

In [100]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(base_estimator = clf, n_estimators = 50, learning_rate = 0.25, random_state = 1)
ada.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=1, splitter='best'),
          learning_rate=0.25, n_estimators=50, random_state=1)

In [101]:
probability = ada.predict_proba(X_train)
predictions = probability[:,1]
score = roc_auc_score(y_train, predictions)
print("Area under ROC {0}".format(score))

Area under ROC 0.9975640246512065


In [102]:
probability = gbm.predict_proba(X_test[features_columns])
predictions = probability[:,1]
score = roc_auc_score(y_test, predictions)
print("Area under ROC {0}".format(score))

Area under ROC 0.9752302013499035


In [103]:
test_probability = ada.predict_proba(test[features_columns])
test_predictions = test_probability[:,1]
output = pd.DataFrame({'activity_id': test_ids, 'outcome':test_predictions})
output.head()

,activity_id,outcome
0,act1_249281,2.220446e-16
1,act2_230855,2.220446e-16
2,act1_240724,1.000000e+00
3,act1_83552,1.000000e+00
4,act2_1043301,1.000000e+00


### Bagging

In [104]:
bag = BaggingClassifier(base_estimator = clf, n_estimators = 50, max_samples = 1.0, max_features = 1.0,
                       bootstrap = True, bootstrap_features = False, n_jobs = 1, random_state = 1)
bag.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=1, splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=50, n_jobs=1, oob_score=False,
         random_state=1, verbose=0, warm_start=False)

In [105]:
probability = bag.predict_proba(X_train)
predictions = probability[:,1]
score = roc_auc_score(y_train, predictions)
print("Area under ROC {0}".format(score))

Area under ROC 0.9975048557900462


In [106]:
probability = bag.predict_proba(X_test[features_columns])
predictions = probability[:,1]
score = roc_auc_score(y_test, predictions)
print("Area under ROC {0}".format(score))

Area under ROC 0.9964264788756138


In [107]:
test_probability = bag.predict_proba(test[features_columns])
test_predictions = test_probability[:,1]
output = pd.DataFrame({'activity_id': test_ids, 'outcome':test_predictions})
output.head()

,activity_id,outcome
0,act1_249281,0.0
1,act2_230855,0.0
2,act1_240724,1.0
3,act1_83552,1.0
4,act2_1043301,1.0


### Extra Tree Classifier

In [108]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier(n_estimators = 200, random_state = 20, n_jobs = -1)
clf.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
           oob_score=False, random_state=20, verbose=0, warm_start=False)

In [109]:
probability = clf.predict_proba(X_train)
predictions = probability[:,1]
score = roc_auc_score(y_train, predictions)
print("Area under ROC {0}".format(score))

Area under ROC 0.9975640246512064


In [110]:
probability = clf.predict_proba(X_test[features_columns])
predictions = probability[:,1]
score = roc_auc_score(y_test, predictions)
print("Area under ROC {0}".format(score))

Area under ROC 0.9965092604465885


In [113]:
test_probability = clf.predict_proba(test[features_columns])
test_predictions = test_probability[:,1]
output = pd.DataFrame({'activity_id': test_ids, 'outcome':test_predictions})
output.head()

,activity_id,outcome
0,act1_249281,0.0
1,act2_230855,0.0
2,act1_240724,1.0
3,act1_83552,1.0
4,act2_1043301,1.0


### XGBoost

In [16]:
import xgboost as xgb

In [15]:
gbm = xgb.XGBClassifier(max_depth=15, n_estimators=150, learning_rate=0.15, colsample_bytree = 0.35)
gbm.fit(X_train, y_train)

KeyboardInterrupt: 

### Training on full Training dataset

In [24]:
gbm = xgb.XGBClassifier(max_depth=15, n_estimators=200, learning_rate=0.054, colsample_bytree = 0.9)
gbm.fit(features[features_columns], labels)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.9,
       gamma=0, learning_rate=0.054, max_delta_step=0, max_depth=15,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [25]:
probability = gbm.predict_proba(features[features_columns])
predictions = probability[:,1]
score = roc_auc_score(labels, predictions)
print("Area under ROC {0}".format(score))

Area under ROC 0.998170163317799


In [26]:
test_probability = gbm.predict_proba(test[features_columns])
test_predictions = test_probability[:,1]
output = pd.DataFrame({'activity_id': test_ids, 'outcome':test_predictions})
output.head()

,activity_id,outcome
0,act1_249281,0.186238
1,act2_230855,0.186238
2,act1_240724,0.927641
3,act1_83552,0.927641
4,act2_1043301,0.927641


In [23]:
features["last_pred"] = predictions
test["last_pred"] = test_predictions

### Validation set

In [142]:
probability = gbm.predict_proba(X_train)
predictions = probability[:,1]
score = roc_auc_score(y_train, predictions)
print("Area under ROC {0}".format(score))

Area under ROC 0.9837044853427995


In [143]:
probability = gbm.predict_proba(X_test[features_columns])
predictions = probability[:,1]
score = roc_auc_score(y_test, predictions)
print("Area under ROC {0}".format(score))

Area under ROC 0.9829993861824684


In [144]:
test_probability = gbm.predict_proba(test[features_columns])
test_predictions = test_probability[:,1]
output = pd.DataFrame({'activity_id': test_ids, 'outcome':test_predictions})
output.head()

,activity_id,outcome
0,act1_249281,0.629073
1,act2_230855,0.629073
2,act1_240724,0.764924
3,act1_83552,0.764924
4,act2_1043301,0.764924


Wow, the training and test AUC are almost same. The model seems to generalize well.

In [27]:
output.to_csv('submission1_10.csv',index = False)